In [71]:
import data
import importlib
importlib.reload(data) #to remive in final .py
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 7it [20:12, 173.23s/it]0:00<?, ?it/s]
LOADING DATA: 7it [00:24,  2.42s/it]                       

In [72]:
import variables
import contraintes
import importlib
importlib.reload(variables) #to remive in final .py
importlib.reload(contraintes) #to remive in final .py

<module 'contraintes' from '/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py'>

In [73]:
def show_solution():
    print(var_dict)
    for machine in var_dict.keys():
        print(machine)
        # for sillon in var_dict[machine].keys():
            # print(var_dict[machine][sillon].X)


In [74]:
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
final_train_task = var_dict["essai de frein départ"][final_train_task]
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)

m.update()
# m.printStats()
# m.presolve()
# m.printStats()
m.optimize()
m.write("test.lp")
show_solution()

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 473.68it/s]
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  1][taches_df["Type de train"] == type_train]["Type de tache humaine"].iloc[0]
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  1][taches_df["Type de train"] == type_train]["Type de tache humaine"].iloc[0]
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  1][taches_df["Type de train"] == type_train]["Type de tache humaine"].iloc[0]
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:104: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  1][taches_df["Type de train"] == type_train]["Type de tache humaine"].iloc[0]
/Users/elliot09/SNCF-Ressources-Optim

AttributeError: 'gurobipy.Model' object has no attribute 'addConst'

In [ ]:
# tests 
importlib.reload(contraintes)
importlib.reload(variables)
# create model
m = Model()

# generate variables (integers)
var_dict = variables.generate_variables(m, dataframes)
# generate variables
contraintes.generate_contraintes(m, dataframes, var_dict)

Initialising variables: 100%|██████████| 217/217 [00:00<00:00, 480.76it/s]
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:103: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:103: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:103: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:103: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:103: UserWarning: Boolean Series key will be reindex

636
Adding occupation constrainst for: WPY_DEP...


/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:172: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles = {}
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:174: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:172: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles = {}
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:174: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles[chantier, "start"] = taches_df[taches_df["Chantier"] == chantier][taches_df.Ordre ==
/Users/elliot09/SNCF-Ressources-Optimisation-/contraintes.py:172: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  chantier_cycles = {}
/Users/elliot09

Adding occupation constrainst for: WPY_FOR...
Adding occupation constrainst for: WPY_REC...
